# Multiple parallel input and Multi-Step output
### Importing


In [3]:
from numpy import array
from numpy import hstack
from keras.models import  Sequential
from keras.layers import LSTM, Dense, TimeDistributed, RepeatVector

### method

In [4]:
def split_sequence(sequence, n_step_in, n_step_out):
   x,y = list(), list() #x is data and y is label
   for i in range(len(sequence)):
       end_ix = i + n_step_in #timestep
       out_end_ix = end_ix + n_step_out # n-output
       if out_end_ix > len(sequence): #In this -if- we want if data be beyond len(sequence) , the command will not continue and break.
          break
       seq_x, seq_y = sequence[i:end_ix,:] , sequence[end_ix:out_end_ix,:]   
       x.append(seq_x)
       y.append(seq_y)
   return array(x) , array(y)

- making data x,y

In [13]:
#n_step_in: timestep or look_up
n_step_in, n_step_out = 3 , 3   

seq1 = array([10,20,30,40,50,60,70,80,90])
seq2 = array([10,20,30,40,50,60,70,80,90])
seq3 = array([10,20,30,40,50,60,70,80,90])   
   
#The input must be three-dimensional or three-channel, that's why we reshape it
seq1 = seq1.reshape((len(seq1),1))
seq2 = seq2.reshape((len(seq2),1))
seq3 = seq3.reshape((len(seq3),1))
   
dataset = hstack((seq1,seq2,seq3))      


x,y = split_sequence(dataset, n_step_in, n_step_out)
print(x.shape,y.shape)   
print('-'*75) #print -----
   
for i in range(len(x)):   
   print(x[i],y[i])


(4, 3, 3) (4, 3, 3)
---------------------------------------------------------------------------
[[10 10 10]
 [20 20 20]
 [30 30 30]] [[40 40 40]
 [50 50 50]
 [60 60 60]]
[[20 20 20]
 [30 30 30]
 [40 40 40]] [[50 50 50]
 [60 60 60]
 [70 70 70]]
[[30 30 30]
 [40 40 40]
 [50 50 50]] [[60 60 60]
 [70 70 70]
 [80 80 80]]
[[40 40 40]
 [50 50 50]
 [60 60 60]] [[70 70 70]
 [80 80 80]
 [90 90 90]]


### training model

In [20]:
n_features = x.shape[2]

#stack lstm
model = Sequential()
model.add(LSTM(100,activation = 'relu',input_shape=(n_step_in,n_features)))   
model.add(RepeatVector(n_step_out))   
model.add(LSTM(50,activation = 'relu',return_sequences=True))   
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam',loss='mse')   
model.fit(x,y,epochs=200,verbose=1)
     

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 4384.0894
Epoch 2/200
1/1 [==============================] - 0s 33ms/step - loss: 4348.0942
Epoch 3/200
1/1 [==============================] - 0s 21ms/step - loss: 4312.1050
Epoch 4/200
1/1 [==============================] - 0s 20ms/step - loss: 4275.5864
Epoch 5/200
1/1 [==============================] - 0s 21ms/step - loss: 4235.7993
Epoch 6/200
1/1 [==============================] - 0s 20ms/step - loss: 4190.2134
Epoch 7/200
1/1 [==============================] - 0s 17ms/step - loss: 4136.6025
Epoch 8/200
1/1 [==============================] - 0s 21ms/step - loss: 4072.8684
Epoch 9/200
1/1 [==============================] - 0s 17ms/step - loss: 3996.7493
Epoch 10/200
1/1 [==============================] - 0s 17ms/step - loss: 3905.2383
Epoch 11/200
1/1 [==============================] - 0s 18ms/step - loss: 3795.5515
Epoch 12/200
1/1 [==============================] - 0s 15ms/step - loss: 3666.3835
Epoch 13/200
1/

### testing

In [23]:
x_input = array([[10,10,10],[20,20,20],[30,30,30]])
x_input = x_input.reshape(1,n_step_in,n_features)
x_input

array([[[10, 10, 10],
        [20, 20, 20],
        [30, 30, 30]]])

In [25]:
yhat = model.predict(x_input,verbose=0)
print(yhat,'\n')
print(y[0])

[[[39.960194 39.417168 39.823704]
  [49.99582  49.648125 50.26038 ]
  [59.922565 60.24822  59.75832 ]]] 

[[40 40 40]
 [50 50 50]
 [60 60 60]]
